In [4]:
import os
import pandas as pd
from tabula.io import read_pdf

In [6]:
# Función para procesar los archivos PDF y combinarlos en un DataFrame categorizado
def pdf_df_x_categoria(ruta_carpeta):
    df_combinado = pd.DataFrame()
    for nombre_archivo in os.listdir(ruta_carpeta):
        if nombre_archivo.endswith(".pdf"):
            ruta_archivo = os.path.join(ruta_carpeta, nombre_archivo)
            categoria = nombre_archivo.split("-")[1].split("_")[0]
            try:
                tablas = read_pdf(ruta_archivo, pages='all', multiple_tables=True)
                for tabla in tablas:
                    tabla['CATEGORIA'] = categoria  # Agregar columna con la categoría
                    df_combinado = pd.concat([df_combinado, tabla], ignore_index=True)
            except Exception as e:
                print(f"Error procesando {nombre_archivo}: {e}")
    return df_combinado

# Función para separar la primera columna según el patrón y ajustar el DataFrame
def separar_primera_columna(df):
    patron = r"(\d{6})\s+([^\d]+)\s+(\d+\.\d+)"
    nuevas_columnas = df.iloc[:, 0].str.extract(patron)
    nuevas_columnas.columns = ['PRACTICA', 'NOMBRE', 'HONORARIOS']
    nuevas_columnas['HONORARIOS'] = nuevas_columnas['HONORARIOS'].astype(float)
    df = df.drop(df.columns[0], axis=1)
    return pd.concat([nuevas_columnas, df], axis=1)

# Función para renombrar y limpiar columnas
def renombrar_y_limpiar_columnas(df):
    try:
        # Renombrar columnas manualmente
        df.columns = ['PRACTICA', 'NOMBRE', 'HONORARIOS', 'col1', 'AYUDANTE', 'GASTOS', 
                      'DESCARTABLE', 'TOTAL', 'CATEGORIA', 'col2']
        # Eliminar columnas innecesarias
        df = df.drop(['col1', 'col2'], axis=1)
    except Exception as e:
        print(f"Error al renombrar/eliminar columnas: {e}")
    return df

# Función para guardar el DataFrame en un archivo Excel
def guardar_en_excel(df, carpeta_origen, ruta_destino):
    nombre_carpeta = os.path.basename(carpeta_origen)
    nombre_archivo = f"SS-Honorarios-{nombre_carpeta}-2024.xlsx"
    ruta_completa = os.path.join(ruta_destino, nombre_archivo)
    if not os.path.exists(ruta_destino):
        os.makedirs(ruta_destino)
    df.to_excel(ruta_completa, index=False)
    print(f"Archivo guardado como: {ruta_completa}")

# Ruta principal y carpeta de destino
ruta_principal = "C:/Users/Usuario/Documents/GitHub/Fusion-de-Datos-PDF/PDF"
ruta_destino_base = "C:/Users/Usuario/Documents/GitHub/Fusion-de-Datos-PDF/PDF-Procesado"

# Procesar todas las subcarpetas
for nombre_carpeta in os.listdir(ruta_principal):
    ruta_carpeta = os.path.join(ruta_principal, nombre_carpeta)
    if os.path.isdir(ruta_carpeta):
        print(f"Procesando carpeta: {nombre_carpeta}")
        
        # Procesar los PDFs y obtener el DataFrame combinado
        df_combinado = pdf_df_x_categoria(ruta_carpeta)
        
        # Aplicar transformación y limpieza
        if not df_combinado.empty:
            df_combinado = separar_primera_columna(df_combinado)
            df_combinado = renombrar_y_limpiar_columnas(df_combinado)
            df_combinado = df_combinado.drop_duplicates(subset=['PRACTICA'], keep='first')
            df_combinado = df_combinado[df_combinado['PRACTICA'].notna()]
        
        # Guardar el DataFrame en la carpeta de destino correspondiente
        ruta_destino = os.path.join(ruta_destino_base, nombre_carpeta)
        guardar_en_excel(df_combinado, ruta_carpeta, ruta_destino)

Procesando carpeta: Abril
Archivo guardado como: C:/Users/Usuario/Documents/GitHub/Fusion-de-Datos-PDF/PDF-Procesado\Abril\SS-Honorarios-Abril-2024.xlsx
Procesando carpeta: Agosto
Archivo guardado como: C:/Users/Usuario/Documents/GitHub/Fusion-de-Datos-PDF/PDF-Procesado\Agosto\SS-Honorarios-Agosto-2024.xlsx
Procesando carpeta: Enero
Archivo guardado como: C:/Users/Usuario/Documents/GitHub/Fusion-de-Datos-PDF/PDF-Procesado\Enero\SS-Honorarios-Enero-2024.xlsx
Procesando carpeta: Febrero
Archivo guardado como: C:/Users/Usuario/Documents/GitHub/Fusion-de-Datos-PDF/PDF-Procesado\Febrero\SS-Honorarios-Febrero-2024.xlsx
Procesando carpeta: Julio
Error al renombrar/eliminar columnas: Length mismatch: Expected axis has 9 elements, new values have 10 elements
Archivo guardado como: C:/Users/Usuario/Documents/GitHub/Fusion-de-Datos-PDF/PDF-Procesado\Julio\SS-Honorarios-Julio-2024.xlsx
Procesando carpeta: Junio
Error al renombrar/eliminar columnas: Length mismatch: Expected axis has 9 elements, ne